In [83]:
%load_ext autoreload
%autoreload 2

from entities.ambient import Ambient

ambient = Ambient(
     name="Ambient Test",
     ambient_images_path=[
          "./assets/img/ambient_1.png", 
          "./assets/img/ambient_2.png", 
          "./assets/img/ambient_3.png", 
          "./assets/img/ambient_4.png"], 
     audio_effect_path= None, #"./assets/audio/Street Sound Effect.mp3",
)

ambient.start()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%load_ext autoreload
%autoreload 2
from entities.car import Car
from entities.stoplight import Stoplight

import pygame
import random

pygame.init()

window_width = 1000
window_height = 1000
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Intersection Simulation")

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0, 100)
GREEN = (0, 255, 0, 100)
YELLOW = (255, 255, 0, 100)
GRAY = (60, 60, 60)

ambient_1 = pygame.image.load('assets/img/ambient_1.png')
ambient_2 = pygame.image.load('assets/img/ambient_2.png')
ambient_3 = pygame.image.load('assets/img/ambient_3.png')
ambient_4 = pygame.image.load('assets/img/ambient_4.png')

try:
    pygame.mixer.init()
    # Set audio effect in loop
    pygame.mixer.music.load('assets/audio/Street Sound Effect.mp3')
    pygame.mixer.music.play(-1)
except pygame.error as e:
    print(f"Warning: {e}")

# Load image, resize it to be 370x370 and put it in the right corner
def new_func(WINDOW_WIDTH, WINDOW_HEIGHT, window, ambient_1, ambient_2, ambient_3, ambient_4):
    ambient_1 = pygame.transform.scale(ambient_1, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
    ambient_2 = pygame.transform.scale(ambient_2, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
    ambient_3 = pygame.transform.scale(ambient_3, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
    ambient_4 = pygame.transform.scale(ambient_4, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
    window.blit(ambient_1, (0, 0))
    window.blit(ambient_2, (WINDOW_WIDTH//2 + 30, 0))
    window.blit(ambient_3, (WINDOW_WIDTH//2 + 30, WINDOW_HEIGHT//2 + 30))
    window.blit(ambient_4, (0, WINDOW_HEIGHT//2 + 30))

new_func(window_width, window_height, window, ambient_1, ambient_2, ambient_3, ambient_4)

   

run = True
clock = pygame.time.Clock()

cars = []
prev_time = 0
stoplight = Stoplight( random.choice([GREEN, RED]) )

while run:

    clock.tick(30)
    stoplight.update_stoplight()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    time = pygame.time.get_ticks() // 1000

    if time != prev_time:
        if time % 2 == 0:
            direction = random.choice(['up', 'down', 'left', 'right'])
            if direction == 'up':
                cars.append(Car(window_width//2 + 5, window_height, direction))
            elif direction == 'down':
                cars.append(Car(window_width//2 - 20 - 4, 0, direction))
            elif direction == 'left':
                cars.append(Car(window_width, window_height//2 - 20 - 4, direction))
            elif direction == 'right':
                cars.append(Car(0, window_height//2 + 5, direction))
        prev_time = time

    if stoplight.time_green >= 300:
        stoplight.switch_yellow()
    
    for car in cars:
        if not car.isStopped:
            if ((car.direction == 'up' and car.y == self.window_height//2 + 50 and (stoplight.color_NS == RED or stoplight.color_NS == YELLOW)) or
                (car.direction == 'down' and car.y + car.length == self.window_height//2 - 50 and (stoplight.color_NS == RED or stoplight.color_NS == YELLOW)) or
                (car.direction == 'left' and car.x == self.window_width//2 + 50 and (stoplight.color_EW == RED or stoplight.color_EW == YELLOW)) or
                (car.direction == 'right' and car.x + car.length == self.window_width//2 - 50 and (stoplight.color_EW == RED or stoplight.color_EW == YELLOW))) or not car.can_move(cars):
                car.stop()
            else:
                if ((car.direction == 'up' and car.y <= self.window_height//2 + car.speed and car.y >= self.window_height//2 - car.speed) or
                    (car.direction == 'down' and car.y + car.length >= self.window_height//2 - car.speed and car.y + car.length <= self.window_height//2 + car.speed) or
                    (car.direction == 'left' and car.x <= self.window_width//2 + car.speed and car.x >= self.window_width//2 - car.speed) or
                    (car.direction == 'right' and car.x + car.length >= self.window_width//2 - car.speed and car.x + car.length <= self.window_width//2 + car.speed)):
                    car.turn_or_straight()
                car.move()
        elif car.isStopped and ((car.direction in ['up', 'down'] and stoplight.color_NS == GREEN) or (car.direction in ['left', 'right'] and stoplight.color_EW == GREEN)):
            car.isStopped = False   # TODO: better to use a method here
            car.move()

        if car.isStopped:
            car.waiting_time += 1
        else:
            car.waiting_time = 0

    cars = [car for car in cars if 0 <= car.x <= window_width and 0 <= car.y <= window_height]

    # Draw intersection
    ## Draw road
    pygame.draw.line(self.window, GRAY, (0, self.window_height//2), (self.window_width, self.window_height//2), 60)
    pygame.draw.line(self.window, GRAY, (self.window_width//2, 0), (self.window_width//2, self.window_height), 60)
    ## Draw lanes
    pygame.draw.line(self.window, WHITE, (0, self.window_height//2 - 28), (self.window_width, self.window_height//2 - 28), 1)
    pygame.draw.line(self.window, WHITE, (0, self.window_height//2 + 28), (self.window_width, self.window_height//2 + 28), 1)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 28, 0), (self.window_width//2 - 28, self.window_height), 1)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 28, 0), (self.window_width//2 + 28, self.window_height), 1)
    pygame.draw.line(self.window, WHITE, (0, self.window_height//2), (self.window_width, self.window_height//2), 4)
    pygame.draw.line(self.window, WHITE, (self.window_width//2, 0), (self.window_width//2, self.window_height), 4)
    ## Draw crosswalks
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 23, self.window_height//2 - 200), (self.window_width//2 - 23, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 17, self.window_height//2 -200), (self.window_width//2 - 17, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 12, self.window_height//2 - 200), (self.window_width//2 - 12, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 6, self.window_height//2 - 200), (self.window_width//2 - 6, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 6, self.window_height//2 - 200), (self.window_width//2 + 6, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 12, self.window_height//2 - 200), (self.window_width//2 + 12, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 17, self.window_height//2 - 200), (self.window_width//2 + 17, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 23, self.window_height//2 - 200), (self.window_width//2 + 23, self.window_height//2 - 180), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 - 23), (self.window_width//2 - 180, self.window_height//2 - 23), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 - 17), (self.window_width//2 - 180, self.window_height//2 - 17), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 - 12), (self.window_width//2 - 180, self.window_height//2 - 12), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 - 6), (self.window_width//2 - 180, self.window_height//2 - 6), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 + 6), (self.window_width//2 - 180, self.window_height//2 + 6), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 + 12), (self.window_width//2 - 180, self.window_height//2 + 12), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 + 17), (self.window_width//2 - 180, self.window_height//2 + 17), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 200, self.window_height//2 + 23), (self.window_width//2 - 180, self.window_height//2 + 23), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 - 23), (self.window_width//2 + 200, self.window_height//2 - 23), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 - 17), (self.window_width//2 + 200, self.window_height//2 - 17), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 - 12), (self.window_width//2 + 200, self.window_height//2 - 12), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 - 6), (self.window_width//2 + 200, self.window_height//2 - 6), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 + 6), (self.window_width//2 + 200, self.window_height//2 + 6), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 + 12), (self.window_width//2 + 200, self.window_height//2 + 12), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 + 17), (self.window_width//2 + 200, self.window_height//2 + 17), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 180, self.window_height//2 + 23), (self.window_width//2 + 200, self.window_height//2 + 23), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 23, self.window_height//2 + 200), (self.window_width//2 - 23, self.window_height//2 + 220), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 17, self.window_height//2 + 200), (self.window_width//2 - 17, self.window_height//2 + 220), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 12, self.window_height//2 + 200), (self.window_width//2 - 12, self.window_height//2 + 220), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 - 6, self.window_height//2 + 200), (self.window_width//2 - 6, self.window_height//2 + 220), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 6, self.window_height//2 + 200), (self.window_width//2 + 6, self.window_height//2 + 220), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 12, self.window_height//2 + 200), (self.window_width//2 + 12, self.window_height//2 + 220), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 17, self.window_height//2 + 200), (self.window_width//2 + 17, self.window_height//2 + 220), 2)
    pygame.draw.line(self.window, WHITE, (self.window_width//2 + 23, self.window_height//2 + 200), (self.window_width//2 + 23, self.window_height//2 + 220), 2)
    ## Cover intersection
    pygame.draw.rect(self.window, GRAY, (self.window_width//2 - 29, self.window_height//2 - 29, 60, 60))
    ## Draw stoplights as stop lines
    pygame.draw.line(self.window, stoplight.color_NS, (self.window_width//2 - 27, self.window_height//2 - 32), (self.window_width//2 - 2, self.window_height//2 - 32), 5)
    pygame.draw.line(self.window, stoplight.color_NS, (self.window_width//2 + 3, self.window_height//2 + 33), (self.window_width//2 + 27, self.window_height//2 + 33), 5)
    pygame.draw.line(self.window, stoplight.color_EW, (self.window_width//2 - 32, self.window_height//2 + 3), (self.window_width//2 - 32, self.window_height//2 + 27), 5)
    pygame.draw.line(self.window, stoplight.color_EW, (self.window_width//2 + 33, self.window_height//2 - 27), (self.window_width//2 + 33, self.window_height//2 - 2), 5)
    
    # Draw cars
    for car in cars:
        car.draw(self.window)
    
    pygame.display.update()

pygame.quit()

[autoreload of entities.stoplight failed: Traceback (most recent call last):
  File "/Users/davidecapone/Library/Python/3.11/lib/python/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/Users/davidecapone/Library/Python/3.11/lib/python/site-packages/IPython/extensions/autoreload.py", line 459, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/davidecapone/Documents/IntersectionMDP/entities/stoplight.py", line 3, in <module>
    from colors import TrafficLightColor
ModuleNotFoundError: No mod

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
